#BERT EMBEDDER'I TRAIN OLMAYACAKSA:

>ADIM 1 -> Datayı tokenize etme. (data preprocessing)

>ADIM 2 -> Bert ile tokenize edilen dataya embedding işlemi uygulama.

>ADIM 3 -> Train edilecek data'yı pipeline'a koyma(train işlemi için gereken veri tipi).

>ADIM 4 -> Embedded olan datayı linear layer ile sınıflandırma. Yanlış sınıflandırırsa loss fonksiyonu ile modeli eğitme.



#BERT EMBEDDER TRAIN OLACAKSA:

> ADIM 1 -> Datayı tokenize etme (data preprocessing)

> ADIM 2 -> Dataya maskeleme işlemi uygulama.

> ADIM 3 -> Train edilecek data'yı pipeline'a koy.(train işlemi için gereken veri tipi).

> ADIM 4 -> Bert embedder'ı tokenize edilmiş ve maskelenen data ile eğit. (Maske ile kapatılan yeri tahmin ederek.)

> ADIM 5 -> Üstte train ettiğimiz Bert embedder sayesinde Embedded olan data'yı,linear layer ile sınıflandırma. Yanlış sınıflandırırsa loss fonksiyonu ile modeli eğitme.             

In [1]:
!pip install transformers

     |████████████████████████████████| 3.3 MB 13.5 MB/s 
     |████████████████████████████████| 3.3 MB 43.0 MB/s 
     |████████████████████████████████| 61 kB 561 kB/s 
     |████████████████████████████████| 596 kB 52.2 MB/s 
     |████████████████████████████████| 895 kB 40.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer
import numpy as np
import torch
import pandas as pd
import time

In [3]:
df_train=pd.read_csv("/content/train.csv",index_col=[0],encoding="windows-1252")
df_test=pd.read_csv("/content/test.csv",index_col=[0],encoding="windows-1252")

In [4]:
#Türkçe cümleleri tokenize etmesi için önceden train edilmiş tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased",do_lower_case=True) 

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/245k [00:00<?, ?B/s]

In [5]:
df_train["Label"].value_counts()
#Dengeli. Dengeleme işlemine gerek yok.

1    3998
0    3998
Name: Label, dtype: int64

In [6]:
comments = df_train.comment.values
labels = df_train.Label.values

#Commentler ve labelların yüklenmesi


In [7]:
from transformers import AutoTokenizer
#Load BERT Turkish tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased",do_lower_case=True)

#Bert bir encoderdır. Yani verilen bir cümleyi, anlamsal bir şekilde embedding
#edilmesini sağlar. Burda, embedding yapalacak cümleleri tokenize etmesi için 
#önceden train edilen bert tokenizer'ı çağırıyorum. Bert tokenizer cümlelere
#anlamlı özel tokenler koyup ("ing","cls","sep"), kelimeleri encode eder. Bu sayede matematiksel olarak, anlamsal bir embedding
#yapılması mümkün hale gelir.




In [8]:
 #Tokenizer'ın kabul ettiği data type olan 'list' e dönüştürme işlemi.
comments=comments.tolist()
indices=tokenizer.batch_encode_plus(comments,max_length=128,add_special_tokens=True, return_attention_mask=True,pad_to_max_length=True,truncation=True) #Encode işlemi (128'e kadar padding ile)
input_ids=indices["input_ids"] #Kelimelerin encodingten sonraki id'leri
attention_masks=indices["attention_mask"] 
print(input_ids[5])
print(comments[5])
print(attention_masks[5])


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[2, 4345, 20718, 18467, 6163, 2636, 2938, 18972, 2262, 25775, 83, 2220, 13382, 1030, 11662, 2016, 2751, 4253, 3930, 29391, 2395, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
bi timsah filmi diyemicem ama yinede o kadar kötü degildi kendi dalina göre iyi. 

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
#Datanın train, test' e ayrılması
from sklearn.model_selection import train_test_split

# Use 99% for training and 1% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=42, test_size=0.2)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=42, test_size=0.2)

In [10]:
# Datayı, pytorchta  train edilmesi için tensorlara dönüştürüyoruz.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels, dtype=torch.long)
validation_labels = torch.tensor(validation_labels, dtype=torch.long)
train_masks = torch.tensor(train_masks, dtype=torch.long)
validation_masks = torch.tensor(validation_masks, dtype=torch.long)

In [11]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32 #Önerilen batch size.

# MLM için (Masked Language Modelling) input pipeline yaratma işlemi:
#Datanın train edilmesi için zorunlu olan aşamadır.  
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Validation set için aynı dataLoader pipeline'ı yaratma.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [12]:
from transformers import AutoModelForSequenceClassification, AdamW, AutoConfig

# Önceden train edilmiş bert modelini yükleme işlemi
config = AutoConfig.from_pretrained(
        "dbmdz/bert-base-turkish-cased",num_labels=2) #. Olumlu veya olumsuz.
model = AutoModelForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased",config=config)

model.cuda()

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [13]:
#Optimizer, model için en iyi parametrelerin seçilmesi için yardımcı olacak. 
#Optimizer'ın hiperparametreleri hiperparametre optimizasyonu ya da deneme  yanılma yöntemiyle seçilebilir.
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  betas=[0.9,0.999],
                  eps = 1e-6 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup

# Epoch sayısı
epochs = 2

# Trainining stepleri
total_steps = len(train_dataloader) * epochs

# Scheduler tasarımı (learning rate optimizasyonu)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [14]:
import numpy as np

# Accuarcy hesaplaması için kullandığım fonksiyon. (Validationda)
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [15]:
import time
import datetime

#Train işleminin süresini ölçmek için fonksiyon.
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [16]:
import torch
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

We will use the GPU: Tesla K80


In [17]:
import random
#Aşağıda kullandığım train metodu, "BertForSequenceClassification" Modelini train etme yöntemidir.
#Label sayısı, üstte config edildiği gibidir. Verilen labellar üzerinden tekrar train
#edilmek üzere, aşşağıda train metodu başlatılmaktadır.

#En baştan halihazırda bulunan bu modeli train etmiyoruz. Ediyor olsaydık, kelimelere
#Maskeleme işlemi yapıp, öyle train ederdik. (Olması gereken cümle ile tahmin edilen cümlenin farkını alıp loss olarak tanımlayarak.)
#Fakat maskeleme yapmadan tüm kelimelere odaklanarak embedding yapıp, linear layer(sonra softmax) ile classification yapıyoruz.

#Attention_mask= Paddingten ve özel tokenlerden dolayı ortaya çıkabilecek odaklanma problemlerini ortadan kaldırır. Örnek:
#Bir cümledeki "gerçek" kelimelere ("ünlem, boşluk vs harici kelimeler") odaklanılması için, attention_mask kullanılır.
#Yani attention_mask, bir maskeleme işlemi değildir. Sadece hangi tokenlere odaklanılması gerektiğini gösterir.
#Burda ise en baştan bert train etmediğimiz için tüm tokenlere odaklanırız.




# Bunu tekrarlanabilir hale getirmek için seed value belirlenir.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# AVG. Lossu plot için kaydetme (her epochta)
loss_values = []

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
   

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Epoch zamanını başlatma
    t0 = time.time()

    # Her bir epoch için loss resetleme
    total_loss = 0

    #Modeli train moduna sokma işlemi.
    model.train()

 
    for step, batch in enumerate(train_dataloader):

        #Her 30 batchte bir ilerleme güncellemesi.
        if step % 30 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        #Bir işlem gerçekleştirmeden önce her zaman önceden hesaplanmış gradyanları temizlenir.
        model.zero_grad()        

        # Forward pass (Loss'u verecektir.)
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
     
        loss = outputs[0]

        #Loss hesabı
        total_loss += loss.item()

        # Gradient hesabı için backward pass.
        loss.backward()

     
        #"exploding gradients" problemini elimine etmek için:
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #Model parametrelerin hiper parametrelere göre değişimlerini gradyanları kullanarak hesap etme işlemi:
        optimizer.step()

        # Learning rate güncellemesi.
        scheduler.step()

    # Ortalama loss
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Daha sonra plot edilebilmesi için lossların kaydedilmesi.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
  

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Modeli evalutation mode'a sokma işlemi
    model.eval()

    # Evaluationun değerlerini tutma. 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

 
    for batch in validation_dataloader:
        
        # Her bir batch'ı GPU'ya ekleme işlemi.
        batch = tuple(t.to(device) for t in batch)
        
        #Dataloaderdan batch'ı extract etme.
        b_input_ids, b_input_mask, b_labels = batch
        
        # Torch. No_grad: Gradyanlar kaydolmuyor(hesaplamıyoruz). Böylece validation çok daha hızlı bir hal almış oluyor.
        with torch.no_grad():        

            #Bu sefer output olarak: modelin ne kadar iyi train oluyor olduğunu bulmak için
            #"model()"in outputu loss değil, Logitler olacak (bir modelin outputunun aktivasyon fonksiyonuna girmeden önceki hali)
            #Amaç modelin iyi train olup olmadığını saptamak. Metod bu şekilde.
            #Örnek: Train set, 1 tahmini yapıyor, fakat doğru olan(ground truth) 0. Train loss ile "train" ediliyor.
            #Train edildikten sonra validation set, yani train tarafından görülmemiş olan data tahmin ediliyor ki,
            #bir bakılsın, az önce train olan model yeni datayı (cümleyi) bir önceki epochtan olduğundan daha iyi tahmin edebiliyor mu?
            #edebiliyorsa model gelişiyor demektir. Gelişiyorsa, train edilmeye devam edilir.
          
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        
        logits = outputs[0]

        # Logitler üzerinden hesaplama yapabilmek için geri CPU ya yüklenmesi
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
      
        # Evaluation accuracy hesabı (Logitler ile tahmin yapılıyor(train edilen modelin görmediği data ile), tahmin, gerçekten ne kadar uzak diye test ediliyor.)
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_accuracy += tmp_eval_accuracy

        # Batch sayısı takibi
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 2 ========
Training...
  Batch    30  of    200.    Elapsed: 0:00:36.
  Batch    60  of    200.    Elapsed: 0:01:13.
  Batch    90  of    200.    Elapsed: 0:01:50.
  Batch   120  of    200.    Elapsed: 0:02:27.
  Batch   150  of    200.    Elapsed: 0:03:04.
  Batch   180  of    200.    Elapsed: 0:03:41.

  Average training loss: 0.37
  Training epoch took: 0:04:06

Running Validation...
  Accuracy: 0.92
  Validation took: 0:00:23

======== Epoch 2 / 2 ========
Training...
  Batch    30  of    200.    Elapsed: 0:00:37.
  Batch    60  of    200.    Elapsed: 0:01:14.
  Batch    90  of    200.    Elapsed: 0:01:51.
  Batch   120  of    200.    Elapsed: 0:02:29.
  Batch   150  of    200.    Elapsed: 0:03:05.
  Batch   180  of    200.    Elapsed: 0:03:42.

  Average training loss: 0.18
  Training epoch took: 0:04:06

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:23

Training complete!


In [18]:
#Test için DATALOADER hazırlığı.
comments1 = df_test.comment.values.tolist()
labels1 = df_test.Label.values



indices1=tokenizer.batch_encode_plus(comments1,max_length=128,add_special_tokens=True, return_attention_mask=True,pad_to_max_length=True,truncation=True)
input_ids1=indices1["input_ids"]
attention_masks1=indices1["attention_mask"]

prediction_inputs1= torch.tensor(input_ids1)
prediction_masks1 = torch.tensor(attention_masks1)
prediction_labels1 = torch.tensor(labels1)

 
batch_size = 32 

# Pipeline ayarlaması.
prediction_data1 = TensorDataset(prediction_inputs1, prediction_masks1, prediction_labels1)
prediction_sampler1 = SequentialSampler(prediction_data1)
prediction_dataloader1 = DataLoader(prediction_data1, sampler=prediction_sampler1, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [19]:
#TEST
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs1)))

# Modeli test moduna sokma.
model.eval()


predictions , true_labels = [], []


for batch in prediction_dataloader1:
  # Add Train için GPU'ya ekleme.
  batch = tuple(t.to(device) for t in batch)
  
  # Dataloader'dan inputları çekme işlemi.
  b_input_ids1, b_input_mask1, b_labels1 = batch
  
  # Gradyanları kaydetmiyoruz(hesaplamıyoruz) ki (validationdaki gibi) test işlemi hızlı gitsin
  with torch.no_grad():
      # Logit hesabı
      outputs1 = model(b_input_ids1, token_type_ids=None, 
                      attention_mask=b_input_mask1)

  logits1 = outputs1[0]

  # Logitleri hesap yapabilmek için CPU'ya geri taşıma.
  logits1 = logits1.detach().cpu().numpy()
  label_ids1 = b_labels1.to('cpu').numpy()
  
  predictions.append(logits1)
  true_labels.append(label_ids1)

print('DONE.')

Predicting labels for 2,666 test sentences...
DONE.


In [20]:
# Test edilecek data ile yapılan tahmin ile gerçek label arasındaki fark'ı bulmak için yapılan hazırlık.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [21]:
#Accuracy hesabı
from sklearn.metrics import accuracy_score
print("Accuracy of BERT is:",accuracy_score(flat_true_labels, flat_predictions))

Accuracy of BERT is: 0.9309827456864216


# **Input alarak duygu analizi:**

In [24]:
#Test için DATALOADER hazırlığı.
comment_list=[]
comment=input("Duygu analizi yapmak istediğiniz cümleyi girin:")
comment_list.append(comment)


indices1=tokenizer.batch_encode_plus(comment_list,max_length=128,add_special_tokens=True, return_attention_mask=True,pad_to_max_length=True,truncation=True)
input_ids1=indices1["input_ids"]
attention_masks1=indices1["attention_mask"]


prediction_inputs1= torch.tensor(input_ids1)
prediction_masks1 = torch.tensor(attention_masks1)



# Pipeline ayarlaması.
prediction_data1 = TensorDataset(prediction_inputs1, prediction_masks1)
prediction_sampler1 = SequentialSampler(prediction_data1)
prediction_dataloader1 = DataLoader(prediction_data1, sampler=prediction_sampler1, batch_size=1)


batch = prediction_dataloader1.batch_sampler.sampler.data_source.tensors

#Ortaya logit çıkarabilmek için batch'ı GPU'ya yükleme.
batch = tuple(t.to(device) for t in batch)
b_input_ids1, b_input_mask1 = batch
with torch.no_grad():
      # Logit hesabı
      outputs1 = model(b_input_ids1, token_type_ids=None, 
                      attention_mask = b_input_mask1)



logits1 = outputs1[0]

logits1 = logits1.detach().cpu().numpy()

pred=np.argmax(logits1)
print(pred)




Duygu analizi yapmak istediğiniz cümleyi girin:Merhaba bugün çok güzel şeyler oldu ama yine de mutsuzum.
0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


# **BERT MODELİNİ SIFIRDAN TRAIN ETME İŞLEMİ**


> Bu ise eğer BERT'i en baştan train etmek istiyorsak. Fakat böyle bir ihtiyacımız yok.Burdaki attention mask indexlerinin içinde "MASK" tokeni de olacaktı.Yani model, maskelenen(kapatılan) kelimeyi tahmin etmeye çalışarak train olacaktı.







In [ ]:
#Burda mask tokeni random %15 valid kelimelere uygulama işlemi(cümlede 100 kelime varsa, 15'ini maskeleyip,
#maskelenen kısımları tahmin etme işlemi(yanlış tahminde loss hesabı yapılıp gradyanlar update edilerek train olacak.))
input_ids = labels.detach().clone()
rand = torch.rand(input_ids.shape)

mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)

for i in range(input_ids.shape[0]):

    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    #Maskeleme işlemi
    input_ids[i, selection] = 3 


NameError: ignored

In [ ]:
epochs = 2

for epoch in range(epochs):
    
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # Gradyanları temizleme. (önceden yaptığımız gibi.)
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        #Loss hesabı.
        loss = outputs.loss
        # Gradyan hesabı için backward pass
        loss.backward()
        # Model parametrelerinin optimize edilmesi.
        optim.step()
       
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())